In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))

In [2]:
import pandas as pd

from mcda.electre.discordance import is_counter_veto_occur, counter_veto_pair, counter_veto_count, counter_veto
from mcda.core.scales import QuantitativeScale, PreferenceDirection
from mcda.core.functions import Threshold

1. Create dataframes

In [3]:
alternative_names = ['fiat', 'vwPolo' , 'nissan' , 'toyota' , 'suzuki', 'ford' ]
criteria_names = ['C1', 'C2', 'C3']
profile_names = ['P1', 'P2', 'P3']

In [4]:
fiat = [100, 1000, 6.0]
vwPolo = [150, 5000, 5.7]
nissan = [110, 4000, 6.8]
toyota = [170, 3000, 5.5]
suzuki = [140, 2000, 7.8]
ford = [120, 6000, 8.0]
cars = pd.DataFrame(
    [fiat, vwPolo, nissan, toyota, suzuki, ford],
    index=alternative_names,
    columns=criteria_names,
)
cars

,C1,C2,C3
fiat,100,1000,6.0
vwPolo,150,5000,5.7
nissan,110,4000,6.8
toyota,170,3000,5.5
suzuki,140,2000,7.8
ford,120,6000,8.0


In [5]:
profile1 = [113, 1200, 7.0]
profile2 = [180, 3500, 6.7]
profile3 = [199, 3400, 8.8]
profiles = pd.DataFrame(
    [profile1, profile2, profile3],
    index=profile_names,
    columns=criteria_names
) 
profiles

,C1,C2,C3
P1,113,1200,7.0
P2,180,3500,6.7
P3,199,3400,8.8


2. Create series with thresholds and specify criteria scales and weights

In [6]:
scales = pd.Series([
    QuantitativeScale(100, 500),
    QuantitativeScale(1000, 6000, PreferenceDirection.MIN),
    QuantitativeScale(0.0, 15.0, PreferenceDirection.MIN)
], index=criteria_names)
scales

C1      Scale [100, 500], max direction
C2    Scale [1000, 6000], min direction
C3     Scale [0.0, 15.0], min direction
dtype: object

In [7]:
cv = pd.Series([
    Threshold(0.3, 20),
    Threshold(0.4, 500),
    None,
], index=criteria_names)
cv

C1     Threshold: alpha=0.3, beta=20
C2    Threshold: alpha=0.4, beta=500
C3                              None
dtype: object

3. Let's begin with something simple :)

Checking if counter veto occurs on the C2 criterion between fiat and suzuki:

In [8]:
is_counter_veto_occur(
    cars['C2']['fiat'],
    cars['C2']['suzuki'],
    scales['C2'],
    cv['C2']
)

False

4. To display if counter-veto occur on all criteria between two alternatives, use `counter_veto_pair()`

Again, between fiat and suzuki:

In [9]:
counter_veto_pair(
    cars.loc['fiat'],
    cars.loc['suzuki'],
    scales,
    cv
)

C1    False
C2    False
C3    False
dtype: bool

5. If you'd like to have a full table with all alternatives comparison (or alternatives-profiles), use `counter_veto()` function

Inside each cell there's a list with all criteria names, on which counter-veto has occurred.

In [10]:
counter_veto(
    cars,
    scales,
    cv
)

,fiat,vwPolo,nissan,toyota,suzuki,ford
fiat,[],[C2],[C2],[C2],[],[C2]
vwPolo,[],[],[],[],[],[]
nissan,[],[],[],[],[],[]
toyota,[C1],[],[C1],[],[],[C2]
suzuki,[],[C2],[],[],[],[C2]
ford,[],[],[],[],[],[]


With profiles:

In [11]:
alt_prof, prof_alt = counter_veto(
    cars,
    scales,
    cv,
    profiles
)

In [12]:
alt_prof

,P1,P2,P3
fiat,[],[C2],[C2]
vwPolo,[],[],[]
nissan,[],[],[]
toyota,[C1],[],[]
suzuki,[],[],[]
ford,[],[],[]


In [13]:
prof_alt

,fiat,vwPolo,nissan,toyota,suzuki,ford
P1,[],[C2],[C2],[C2],[],[C2]
P2,[C1],[],[C1],[],[],[C1]
P3,[C1],[],[C1],[],[],[C1]


6. Additionally, there's another function to use, called `counter_veto_count()`. Input is the same as in the previous function, but as an output it returns table with counted counter-veto occurrences.

In [14]:
counter_veto_count(
    cars,
    scales,
    cv
)

,fiat,vwPolo,nissan,toyota,suzuki,ford
fiat,0,1,1,1,0,1
vwPolo,0,0,0,0,0,0
nissan,0,0,0,0,0,0
toyota,1,0,1,0,0,1
suzuki,0,1,0,0,0,1
ford,0,0,0,0,0,0


6. Concordance table between alternatives and profiles

In [15]:
alt_prof, prof_alt = counter_veto_count(
    cars,
    scales,
    cv,
    profiles,
)

In [16]:
alt_prof

,P1,P2,P3
fiat,0,1,1
vwPolo,0,0,0
nissan,0,0,0
toyota,1,0,0
suzuki,0,0,0
ford,0,0,0


In [17]:
prof_alt

,fiat,vwPolo,nissan,toyota,suzuki,ford
P1,0,1,1,1,0,1
P2,1,0,1,0,0,1
P3,1,0,1,0,0,1
